<a href="https://colab.research.google.com/github/IsraaAbdulNabi/Weather-Forecasting-Using-Machine-Learning-and-DeepLearning-Techniques-A-Case-Study-in-Jordan/blob/main/Weather_Forecasting_using_ML_and_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipaddress
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
from datetime import datetime, timedelta
import math
import missingno as msno
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore') 
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import preprocessing
import torch
from sklearn import svm
from sklearn import tree
from sklearn.externals import joblib
import pickle
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
df = pd.read_csv("weather_data_24hr.csv")

In [ ]:
df.info()

In [ ]:
#Remove any repatative representation of data in fahrenheit vs. celsius and Miles vs. Km
df= df.drop(['maxtempF', 'mintempF', 'avgtempF', 'totalprecipIn',
             'weatherCode', 'weatherIconUrl', 'visibilityMiles', 
             'HeatIndexF', 'DewPointF','WindChillF', 'WindGustMiles',
             'FeelsLikeF','windspeedMiles','weatherDesc','moon_phase','winddir16point'],axis=1)

In [ ]:
df['sunrise'] = pd.to_datetime(df['sunrise'],format='%H:%M %p')
df['sunset'] = pd.to_datetime(df['sunset'],format='%H:%M %p')
df.drop(df[df['moonrise'] == 'No moonrise'].index, inplace = True)
df.drop(df[df['moonset'] == 'No moonset'].index, inplace = True)
df['moonrise'] = pd.to_datetime(df['moonrise'],format='%H:%M %p')
df['moonset'] = pd.to_datetime(df['moonset'],format='%H:%M %p')
#Convert datetime to numeric value so we can calculate the correlation between it and the tempreture
df['sunrise'] = df['sunrise'].apply(lambda x: x.value)
df['sunset'] = df['sunset'].apply(lambda x: x.value)
df['moonrise'] = df['moonrise'].apply(lambda x: x.value)
df['moonset'] = df['moonset'].apply(lambda x: x.value)

In [ ]:
df.corr()[['avgtempC']].sort_values('avgtempC')

In [ ]:
import pandas as pd
import numpy as np
X = df.drop(['avgtempC','date'],axis=1)
y = df['avgtempC']
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
X = df.drop(['loc_id','date'],axis=1)
y = df['avgtempC']
#get correlations of each features in dataset
corrmat = X.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(13,10))
#plot heat map
g=sns.heatmap(X[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
from pandas import read_csv
from matplotlib import pyplot
series = read_csv("weather_data_24hr.csv")
series['date'] = pd.to_datetime(series['date'],format='%Y-%m-%d')
series= series[['avgtempC','date']]
series.plot(x='date',y='avgtempC',style='k.')
pyplot.show()

In [ ]:
df = df.drop(['sunrise','pressureMB','cloudcover','humidity',
               'totalprecipMM','WindGustKmph','windspeedKmph',
               'moon_illumination','moonrise','moonset','visibilityKm',
               'sunhour','winddirdegree','pressureInches'],axis=1)

In [ ]:
amman_df= df.loc[df['loc_id'] == 1]
amman_df=amman_df.set_index('date')
amman_df = amman_df.drop(['loc_id'],axis=1)
irbid_df= df.loc[df['loc_id'] == 2]
irbid_df=irbid_df.set_index('date')
irbid_df = irbid_df.drop(['loc_id'],axis=1)
aqaba_df= df.loc[df['loc_id'] == 3]
aqaba_df=aqaba_df.set_index('date')
aqaba_df = aqaba_df.drop(['loc_id'],axis=1)

In [ ]:
#set of selected features
features = ['avgtempC','WindChillC','HeatIndexC','FeelsLikeC','uvIndex',
            'sunset', 'DewPointC']

In [ ]:
def derive_nth_day_feature(df, feature, N):
    rows = df.shape[0]
    nth_prior_measurements = [None]*N + [df[feature][i-N] for i in range(N, rows)]
    col_name = "{}_{}".format(feature, N)
    df[col_name] = nth_prior_measurements

In [ ]:
for feature in features:
    if feature != 'date':
        for N in range(1, 4):
            derive_nth_day_feature(amman_df, feature, N)

In [ ]:
amman_df.head(10)

In [ ]:
for feature in features:
    if feature != 'date':
        for N in range(1, 4):
            derive_nth_day_feature(irbid_df, feature, N)

In [ ]:
for feature in features:
    if feature != 'date':
        for N in range(1, 4):
            derive_nth_day_feature(aqaba_df, feature, N)

In [ ]:
amman_df = amman_df.dropna()
irbid_df = irbid_df.dropna()
aqaba_df = aqaba_df.dropna()

In [ ]:
amman_df.to_csv("amman_waether_data_24h.csv")
irbid_df.to_csv("irbid_waether_data_24h.csv")
aqaba_df.to_csv("aqaba_waether_data_24h.csv")

In [ ]:
amman_df.corr()[['avgtempC']].sort_values('avgtempC')

In [ ]:
amman_df=amman_df.drop(['mintempC','maxtempC'],axis=1)
irbid_df=irbid_df.drop(['mintempC','maxtempC'],axis=1)
aqaba_df=aqaba_df.drop(['mintempC','maxtempC'],axis=1)

In [ ]:
amman_df.columns

In [ ]:
from pycaret.regression import *
exp_reg101 = setup(data = df, target = 'avgtempC', session_id=123)

In [ ]:
best = compare_models(exclude = ['ransac'])

In [ ]:
#Using SciKit-Learn's LinearRegression Module to Predict the Weather
from sklearn.model_selection import train_test_split
# Data Partition
X_amman = amman_df.drop('avgtempC', axis=1)
y_amman = amman_df['avgtempC']

X_train, X_test, y_train, y_test = train_test_split(X_amman, y_amman, test_size=0.3, random_state=23,shuffle= False)

In [ ]:
from sklearn.linear_model import LinearRegression
# instantiate the regressor class
model = LinearRegression()

# fit the build the model by fitting the regressor to the training data
model.fit(X_train, y_train)

# make a prediction set using the test set
prediction = model.predict(X_test)

# Evaluate the prediction accuracy of the model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
rmse = np.sqrt(mean_squared_error(y_test,prediction))
r2 = r2_score(y_test,prediction)
print("RMSE Score for Test set: " +"{:.2}".format(rmse))
print("R2 Score for Test set: " +"{:.2}".format(r2))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
reg=DecisionTreeRegressor(random_state=0)
reg.fit(X_train,y_train)
prediction = reg.predict(X_test)
# evaluate the model

rmse = np.sqrt(mean_squared_error(y_test,prediction))
r2 = r2_score(y_test,prediction)

print("RMSE Score for Test set: " +"{:.2}".format(rmse))
print("R2 Score for Test set: " +"{:.2}".format(r2))


In [ ]:
#Polynomial Regression
from sklearn.preprocessing import PolynomialFeatures

poly_df = PolynomialFeatures(degree = 3)
transform_poly = poly_df.fit_transform(X_train)

linreg2 = LinearRegression()
linreg2.fit(transform_poly,y_train)

polynomial_predict = linreg2.predict(transform_poly)
rmse = np.sqrt(mean_squared_error(y_train,polynomial_predict))
r2 = r2_score(y_train,polynomial_predict)
print("RMSE Score for Test set: " +"{:.2}".format(rmse))
print("R2 Score for Test set: " +"{:.2}".format(r2))

In [ ]:
#Random Forest Model
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=5, random_state=0)
rf_reg.fit(X_train,y_train)
rf_predict = rf_reg.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train,rf_predict))
r2 = r2_score(y_train,rf_predict)


print("RMSE Score for Test set: " +"{:.2}".format(rmse))
print("R2 Score for Test set: " +"{:.2}".format(r2))

In [ ]:
X_train = np.reshape(X_train.values,(3054,27,1))
X_test = np.reshape(X_test.values,(1309,27,1))


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dropout
model = keras.Sequential()

model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mse')

In [ ]:
historay = model.fit(X_train,y_train,validation_split=0.2, epochs=5, batch_size=32)

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

In [ ]:
Y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test,Y_pred))
r2 = r2_score(y_test,Y_pred)

print("RMSE Score for Test set: " +"{:.2}".format(rmse))
print("R2 Score for Test set: " +"{:.2}".format(r2))